In [149]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv 
import edgar
import os
import pickle
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

Get CIK list of all investment companies

In [323]:
df_MF = pd.read_csv(r'https://goz39a.s3.eu-central-1.amazonaws.com/13F/CIK_list.csv')
df_MF['Entity Name'] = df_MF['Entity Name']
#[i.find(isin(t)) for i in df_MF.loc[df_MF['Entity Name']]]
df_MF

,Unnamed: 0,CIK Number,Entity Name,Address_1,City,State,Zip Code
0,0,2110,COLUMBIA ACORN TRUST,227 W MONROE STE 3000,CHICAGO,IL,60606
1,1,2646,Voya INTERMEDIATE BOND PORTFOLIO,ING FUNDS SERVICES,SCOTTSDALE,AZ,85258
2,2,2663,VOYA GOVERNMENT MONEY MARKET PORTFOLIO,"7337 E. DOUBLETREE RANCH ROAD, STE 100",SCOTTSDALE,AZ,85258
3,3,2664,Voya VARIABLE FUNDS,"7337 E. DOUBLETREE RANCH ROAD, STE 100",SCOTTSDALE,AZ,85258
4,4,2691,LORD ABBETT AFFILIATED FUND INC,90 HUDSON STREET,JERSEY CITY,NJ,07302
...,...,...,...,...,...,...,...
2146,2146,1810747,Simplify Exchange Traded Funds,154 W 14TH ST,NEW YORK,NY,10011
2147,2147,1811168,2nd Vote Funds,PO BOX 2916,HENDERSONVILLE,TN,37077
2148,2148,1815558,Waycross Independent Trust,"C/O ULTIMUS FUND SOLUTIONS, LLC",CINCINNATI,OH,45246
2149,2149,1815608,Modern Capital Funds Trust,1050 JOHNNIE DODDS BLVD,MT. PLEASANT,SC,29465


This will be used to link the CIK number to the company names

In [497]:
df_MF['Entity Name'].loc[df_MF['CIK Number'] == 2110].values[0]

'COLUMBIA ACORN TRUST'

download all filings from 2020-2021

In [3]:
path= r'c:/Users/michi/Desktop/filing_history'
edgar.download_index(path, 2020, skip_all_present_except_last=False)

select the URL to the edgar archive for the 13F-HR filings of 2020 until q1 of 2021 

In [4]:
def get_urls(path_download_index):
    with open(path_download_index,'r') as file:
        tsvreader = csv.reader(file, delimiter="|")
        urls= []
        file = []
        for line in tsvreader:
            if line[1:][1] == '13F-HR':
                adress=line[-1]
                url =  'https://www.sec.gov/Archives/' + adress
                holding = line[1]
                urls.append((holding.lower(), url))
    return urls

In [5]:
path= r'c:/Users/michi/Desktop/filing_history'

urls_2020_1 = get_urls(path + os.sep + '2020-QTR1.tsv')
urls_2020_2 = get_urls(path + os.sep + '2020-QTR2.tsv')
urls_2020_3 = get_urls(path + os.sep + '2020-QTR3.tsv')
urls_2020_4 = get_urls(path + os.sep + '2020-QTR4.tsv')
urls_2021 = get_urls(path + os.sep + '2021-QTR1.tsv')

Select only investment companies with 13F-HR filings in all quarters

In [6]:

urls_2020_1names, _ = zip(*urls_2020_1)
urls_2020_2names, _ = zip(*urls_2020_2)
urls_2020_3names, _ = zip(*urls_2020_3)
urls_2020_4names, _ = zip(*urls_2020_4)
urls_2021_names, _ =zip(*urls_2021)

companiesBoth = [i for i in (urls_2020_1names and urls_2020_2names and urls_2020_3names and urls_2020_4names and urls_2021_names  )]


In [7]:
len(companiesBoth)

5970

List of biggest investment companies from https://mutualfunddirectory.org/latest-directory-ranking-here/


In [8]:

URL = 'https://mutualfunddirectory.org/latest-directory-ranking-here/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_="tablepress-id-109")
for company in table.find_all('tbody'):
    rows= company.find_all('tr')
    companies =[]
    for row in rows:
        companies.append(row.find('td', class_="column-2").text)

companies = companies[:-1]
companies = [i.lower() for i in companies]


From companies with filings in each quarter, only select biggest companies 

In [35]:
urls_list=[]
period = [urls_2020_1, urls_2020_2, urls_2020_3, urls_2020_4, urls_2021]
for quarter in period:
    urls = []
    companies_quarter = [i for i in quarter if i[0] in companiesBoth]
    for company_quarter in companies_quarter:
        if len(company_quarter[0].split())>1:
            if company_quarter[0].split()[0] in companies or company_quarter[0].split()[1] in companies:
                urls.append(company_quarter[1])
        else:
             if company_quarter[0].split()[0] in companies:
                urls.append(company_quarter[1])
    urls_list.append(urls)

In [79]:
unique_companies=[urls_list[1][j].split('data/')[1].split('/')[0] for j in range(len(urls_list[1]))]
unique_urls_list=[]
for i in range(len(urls_list)):
    unique_urls=[urls_list[i][j] for j in range(len(urls_list[i])) if urls_list[i][j].split('data/')[1].split('/')[0] in unique_companies]
    unique_urls_list.append(unique_urls)

In [84]:
[print('number of filings: %s'%len(set(unique_urls_list[i]))) for i in range(len(unique_urls_list))]

number of filings: 20
number of filings: 20
number of filings: 20
number of filings: 20
number of filings: 20


[None, None, None, None, None]

In [111]:
unique_urls_list[0]

['https://www.sec.gov/Archives/edgar/data/1027796/0001567619-20-002400-index.html',
 'https://www.sec.gov/Archives/edgar/data/102909/0001104659-20-020869-index.html',
 'https://www.sec.gov/Archives/edgar/data/1109448/0001532155-20-000112-index.html',
 'https://www.sec.gov/Archives/edgar/data/1123274/0001599576-20-000002-index.html',
 'https://www.sec.gov/Archives/edgar/data/1132716/0001132716-20-000003-index.html',
 'https://www.sec.gov/Archives/edgar/data/1291274/0000950123-20-001528-index.html',
 'https://www.sec.gov/Archives/edgar/data/1454937/0001454937-20-000002-index.html',
 'https://www.sec.gov/Archives/edgar/data/1502149/0001567619-20-002196-index.html',
 'https://www.sec.gov/Archives/edgar/data/1529735/0001628280-20-001602-index.html',
 'https://www.sec.gov/Archives/edgar/data/1535602/0001535602-20-000001-index.html',
 'https://www.sec.gov/Archives/edgar/data/1535631/0001535631-20-000001-index.html',
 'https://www.sec.gov/Archives/edgar/data/1599469/0001599469-20-000001-index.

some companies have more than 1 filing in a quarter


Form filing detail page on Edgar scrape the html table

In [138]:

def parser(urls):
    '''Returns dataframe of all the securities of all filings by concatenating the filing tables found at Edgar Archives'''
    df = []
    for url in urls:
        my_header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'
,
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'}
        #url = 'https://www.sec.gov/Archives/edgar/data/1000742/0001567619-20-003961-index.html'
        page = requests.get(url,  headers=my_header)    
        data = page.text
        soup = BeautifulSoup(data, "lxml")
        result= soup.find_all('div', attrs={'class':'formGrouping'})[0]
        date = result.find('div',attrs={'class':'info'}).text
        index = soup.find('div',{'id':'secNum'}).text.replace('SEC Accession No. ','').replace('\n','')
        entityName = soup.find('span',attrs={'class':"companyName"}).text 
        entityName = entityName.partition("(Filer)")[0]

        #get third hyperlink on page which directs to info table
        table_link = soup.find('table',attrs={'class':'tableFile'})
        table_link = table_link.find_all('a')
        table_link = table_link[2]
        identifier=table_link.get('href')
        url = 'https://www.sec.gov'+identifier
        
        #get CIK number from URL
        cik_company = url.split('data/')[1].split('/')[0]
        
        #parse html table to dataframe
        ## retry when 'https forbidden' error is raised, until page can be scraped successfully 
        DF_13F = None
        while DF_13F is None:
            try:
                DF_13F = pd.read_html(url)
                DF_13F = DF_13F[-1]
                DF_13F = DF_13F.iloc[2:]
                new_header = DF_13F.iloc[0]
                DF_13F.columns = new_header
                DF_13F = DF_13F.iloc[2:]
                DF_13F['date_reported'] = date
                DF_13F['CIK'] = cik_company
                DF_13F['filing index'] = index
                DF_13F['Entity name'] = entityName

                # Get entity name from CIK code from list of all funds
                #DF_13F['Entity Name'] = df_MF['Entity Name'].loc[df_MF['CIK Number'] == cik_company].values

                DF_13F = DF_13F[['filing index','date_reported','NAME OF ISSUER','TITLE OF CLASS','CUSIP','(x$1000)','PRN AMT','PRN','CIK','Entity name']]
                df.append(DF_13F)
                print('Filing found for cik %s at %s' %(cik_company,url))

            except:
                print('Retry, could not parse cik %s at: %s'%(cik_company,url))
                pass
    return pd.concat(df)

Repeat parsing in tables for each quarter

In [140]:
df_All = {'quarter_' +str(i) : parser(unique_urls_list[i]) for i in range(4) } 


Filing found for cik 1027796 at https://www.sec.gov/Archives/edgar/data/1027796/000156761920002400/xslForm13F_X01/form13fInfoTable.xml
Filing found for cik 102909 at https://www.sec.gov/Archives/edgar/data/102909/000110465920020869/xslForm13F_X01/infotable.xml
Retry, could not parse cik 1109448 at: https://www.sec.gov/Archives/edgar/data/1109448/000153215520000112/xslForm13F_X01/ACMLInfoTable.xml
Filing found for cik 1109448 at https://www.sec.gov/Archives/edgar/data/1109448/000153215520000112/xslForm13F_X01/ACMLInfoTable.xml
Filing found for cik 1123274 at https://www.sec.gov/Archives/edgar/data/1123274/000159957620000002/xslForm13F_X01/US13F_PCO_EUR_2019_Q4.xml
Filing found for cik 1132716 at https://www.sec.gov/Archives/edgar/data/1132716/000113271620000003/xslForm13F_X01/ubsocQ4_19infotable.xml
Filing found for cik 1291274 at https://www.sec.gov/Archives/edgar/data/1291274/000095012320001528/xslForm13F_X01/form13fInfoTable.xml
Filing found for cik 1454937 at https://www.sec.gov/Arc

Filing found for cik 1502149 at https://www.sec.gov/Archives/edgar/data/1502149/000156761920015059/xslForm13F_X01/form13fInfoTable.xml
Filing found for cik 1529735 at https://www.sec.gov/Archives/edgar/data/1529735/000162828020012721/xslForm13F_X01/mimq22020informationtable.xml
Filing found for cik 1535602 at https://www.sec.gov/Archives/edgar/data/1535602/000153560220000004/xslForm13F_X01/US13F_BPSA_2020_Q2.xml
Filing found for cik 1535631 at https://www.sec.gov/Archives/edgar/data/1535631/000153563120000005/xslForm13F_X01/US13F_PBT_2020_Q2.xml
Filing found for cik 1599469 at https://www.sec.gov/Archives/edgar/data/1599469/000159946920000005/xslForm13F_X01/US13F_BPCAL_2020_Q2.xml
Retry, could not parse cik 1599576 at: https://www.sec.gov/Archives/edgar/data/1599576/000159957620000004/xslForm13F_X01/US13F_PNAA_2020_Q2.xml
Retry, could not parse cik 1599576 at: https://www.sec.gov/Archives/edgar/data/1599576/000159957620000004/xslForm13F_X01/US13F_PNAA_2020_Q2.xml
Filing found for cik 1

In [195]:
# save dataframe object as pickle to use in other notebooks
with open("C:/Users/michi/Desktop/MDA/df_All","wb") as f:
    pickle.dump(df_All, f)
f.close()




In [143]:
df=pickle.load(open('df_All',"rb"))

In [142]:
df_All

{'quarter_0': 2                  filing index date_reported          NAME OF ISSUER  \
 4    0001567619-20-002400          2020-02-07         ACTUANT CORP- A   
 5    0001567619-20-002400          2020-02-07                   AECOM   
 6    0001567619-20-002400          2020-02-07                AES CORP   
 7    0001567619-20-002400          2020-02-07           ALLSTATE CORP   
 8    0001567619-20-002400          2020-02-07              AMDOCS LTD   
 ..                          ...           ...                     ...   
 190  0000949509-20-000002          2020-02-14    WESTERN DIGITAL CORP   
 191  0000949509-20-000002          2020-02-14  WHITING PETROLEUM CORP   
 192  0000949509-20-000002          2020-02-14       YETI HOLDINGS INC   
 193  0000949509-20-000002          2020-02-14                  YPF SA   
 194  0000949509-20-000002          2020-02-14        ZILLOW GROUP INC   
 
 2      TITLE OF CLASS      CUSIP (x$1000)   PRN AMT  PRN      CIK  \
 4                 COM  005

In [147]:
# select first quarter 2020
df = df['quarter_0']

In [25]:
df

2,filing index,date_reported,NAME OF ISSUER,TITLE OF CLASS,CUSIP,(x$1000),PRN AMT,PRN,CIK,Entity name
4,0001567619-20-002400,2020-02-07,ACTUANT CORP- A,COM,00508X203,103856,3989841,SH,1027796,PZENA INVESTMENT MANAGEMENT LLC
5,0001567619-20-002400,2020-02-07,AECOM,COM,00766T100,113169,2623898,SH,1027796,PZENA INVESTMENT MANAGEMENT LLC
6,0001567619-20-002400,2020-02-07,AES CORP,COM,00130H105,909,45692,SH,1027796,PZENA INVESTMENT MANAGEMENT LLC
7,0001567619-20-002400,2020-02-07,ALLSTATE CORP,COM,020002101,31883,283530,SH,1027796,PZENA INVESTMENT MANAGEMENT LLC
8,0001567619-20-002400,2020-02-07,AMDOCS LTD,COM,G02602103,949,13152,SH,1027796,PZENA INVESTMENT MANAGEMENT LLC
...,...,...,...,...,...,...,...,...,...,...
190,0000949509-20-000002,2020-02-14,WESTERN DIGITAL CORP,CONVERTIBLE BOND,958102AP0,8875,9016000,PRN,949509,OAKTREE CAPITAL MANAGEMENT LP
191,0000949509-20-000002,2020-02-14,WHITING PETROLEUM CORP,CONVERTIBLE BOND,966387AL6,5954,6069000,PRN,949509,OAKTREE CAPITAL MANAGEMENT LP
192,0000949509-20-000002,2020-02-14,YETI HOLDINGS INC,COMMON STOCK,98585X104,14533,417866,SH,949509,OAKTREE CAPITAL MANAGEMENT LP
193,0000949509-20-000002,2020-02-14,YPF SA,COMMON STOCK,984245100,15709,1356600,SH,949509,OAKTREE CAPITAL MANAGEMENT LP


In [89]:
# entity names as labels
labels = list(set(df['Entity name'].str.lower()))


In [88]:
labels

{0: 'conning inc. ',
 1: 'pzena investment management llc ',
 2: 'vanguard group inc ',
 3: 'ubs asset management americas inc ',
 4: 'alliancebernstein l.p. ',
 5: 'barings llc ',
 6: 'transamerica financial advisors, inc. ',
 7: 'aegon usa investment management, llc ',
 8: 'ubs group ag ',
 9: 'banque pictet & cie sa ',
 10: 'ubs oconnor llc ',
 11: 'invesco senior secured management inc /adv ',
 12: 'invesco ltd. ',
 13: 'pictet asset management ltd ',
 14: 'pictet north america advisors sa ',
 15: 'vanguard capital wealth advisors ',
 16: 'oaktree capital management lp ',
 17: 'bank pictet & cie (asia) ltd ',
 18: 'pictet & cie (europe) sa ',
 19: 'metlife investment management, llc ',
 20: 'pictet bank & trust ltd '}

Adjacency matrix of positions for holdings

In [26]:
def Adj_count(df):
    dummies = pd.get_dummies(df['CUSIP'])
    df_=pd.concat([df[['CIK']],dummies],axis=1)
    u = df_.groupby(['CIK']).sum()
    v=np.dot(u,u.T)
    # only lower triangle of matrix and diagonal=0
    v=np.tril(v,-1)
    return v

In [27]:
v

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [169,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [150,  52,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [ 26, 101, 160,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [  9, 229, 177,  21,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [139, 235, 246, 225, 170,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [  0,  29,  10,   0,   1,   2,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [ 11,  66,  83,  31,   9, 108,   1,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0],
       [118, 151, 108, 1

In [146]:
def Adj_weight(df['quarter_0']):
    dummies = pd.get_dummies(df['CUSIP']).astype(float)
    weights=dummies.T*np.asarray(df['(x$1000)']).astype(float)
    df_ = pd.concat([df[['CIK']],weights.T],axis=1)
    weights = df_.groupby(['CIK']).sum()
    v=np.dot(weights,weights.T)
    v=np.tril(v,0)
    return v

SyntaxError: invalid syntax (<ipython-input-146-85e351d19780>, line 1)

In [148]:
adj = Adj_weight(df)

In [56]:
with open("C:/Users/michi/Desktop/MDA/adj","wb") as f:
    pickle.dump(adj, f)
f.close()

adj=pickle.load(open('adj',"rb"))

In [7]:
adj.shape

(21, 21)

## Graph

In [150]:
np.random.seed(3)
G = nx.from_numpy_matrix(adj)
labels={i: labels_[i] for i in G.nodes}
#G=nx.relabel_nodes(G, labels)
pos= nx.draw_spring(G,with_labels=True)

plt.axis("off")
plt.show()

NameError: name 'labels_' is not defined

In [52]:
G.edges(data=True)

EdgeDataView([('1109448', '1123274', {'weight': 935663383419.0}), ('1109448', '1132716', {'weight': 214275156760000.0}), ('1109448', '1291274', {'weight': 31726773498037.0}), ('1109448', '1502149', {'weight': 186824843.0}), ('1109448', '1529735', {'weight': 10910036359330.0}), ('1109448', '1535602', {'weight': 7538407550449.0}), ('1109448', '1535631', {'weight': 257992019258.0}), ('1109448', '1599469', {'weight': 307582758889.0}), ('1109448', '1599576', {'weight': 866987726087.0}), ('1109448', '1610520', {'weight': 207233220186437.0}), ('1109448', '1730578', {'weight': 95336004841.0}), ('1109448', '801051', {'weight': 1468743279385.0}), ('1109448', '861177', {'weight': 1.6341347851358333e+17}), ('1109448', '870481', {'weight': 4532116663.0}), ('1109448', '9015', {'weight': 2022134047536.0}), ('1109448', '914208', {'weight': 161751820722637.0}), ('1109448', '949509', {'weight': 85315687167.0}), ('1123274', '1132716', {'weight': 2018970069000.0}), ('1123274', '1291274', {'weight': 173391

In [60]:
bb = nx.betweenness_centrality(G)
cc = nx.closeness_centrality(G)
dc = nx.degree_centrality(G)
centrality = {j:{'betweenness':bb[j],'closeness': cc[j],'degree': dc[j]} for i,j in enumerate(G.nodes)}
nx.set_node_attributes(G, centrality)



dict(sorted(bb.items(), key=lambda item: -item[1]))
# TODO community detection, node connectivity, diagonal of network, eigenvector_centrality, Jaccard index (similarity of every pair)

{1: 0.005729066255382043,
 2: 0.005729066255382043,
 4: 0.005729066255382043,
 5: 0.005729066255382043,
 8: 0.005729066255382043,
 13: 0.005729066255382043,
 15: 0.005729066255382043,
 16: 0.005729066255382043,
 18: 0.005729066255382043,
 19: 0.005729066255382043,
 20: 0.00499807210333526,
 0: 0.003328834907782276,
 14: 0.002692629008418482,
 7: 0.002400231347599768,
 6: 0.0011358524516419254,
 17: 0.0003759398496240601,
 3: 0.0002923976608187134,
 9: 0.0002923976608187134,
 10: 0.0002923976608187134,
 11: 0.0002923976608187134,
 12: 0.0002923976608187134}

In [107]:
summary=pd.DataFrame({'betweenness':bb, 'closeness':cc, 'degree':dc}).reset_index()

In [115]:
summary['index']=labels_

In [116]:
summary

,index,betweenness,closeness,degree
0,conning inc.,0.003329,0.952381,1.05
1,pzena investment management llc,0.005729,1.000000,1.10
2,vanguard group inc,0.005729,1.000000,1.10
3,ubs asset management americas inc,0.000292,0.909091,1.00
4,alliancebernstein l.p.,0.005729,1.000000,1.10
5,barings llc,0.005729,1.000000,1.10
6,"transamerica financial advisors, inc.",0.001136,0.769231,0.80
7,"aegon usa investment management, llc",0.002400,0.909091,1.00
8,ubs group ag,0.005729,1.000000,1.10
9,banque pictet & cie sa,0.000292,0.909091,1.00
